In [1]:
#run in sch_man_nwinvasion-jupyter environment

import os
import shutil
import allel
import math
import yaml
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
#from scipy import stats

In [5]:
os.chdir("/master/nplatt/sch_man_nwinvasion")

In [3]:
with open('data/pop_assign.yml') as yaml_file:
    pop_assign = yaml.load(yaml_file, Loader=yaml.FullLoader)

read in the vcf file and get pop specific allele counts

In [8]:
#-------------------------------------------------------------------------------
# get genotype info per population

#read in vcf
filtered_callset=allel.read_vcf('results/variant_filtration/smv7_ex_autosomes.vcf')

#now get an index for each sample/population
samples = filtered_callset["samples"]

i=0
pop_idxs = defaultdict(list)   
for sample in samples:  
    pop_idxs[pop_assign[sample]].append(i) 
    i=i+1

pops= list(pop_idxs.keys()) 

#get genotypes
gt=allel.GenotypeArray(filtered_callset['calldata/GT'])

#now get allele count per population
ac=gt.count_alleles()

pop_ac={}
for pop in pops:
    pop_ac[pop] = gt.count_alleles(subpop=pop_idxs[pop])

Find all of the accessible bases (since used probes) and get an idea of the length of each chr

In [9]:
#initialize an list the lenght of each contig to fale
accessible_bases = {}
chrom_length = {}

with open('data/genomes/Smansoni_v7.fa.fai', 'r') as fai:
    for entry in fai:
        chrom, length, *offset = entry.rstrip().split("\t")
        chrom_length[chrom] = int(length)
        accessible_bases[chrom]=[False] * int(length)


#now read the bed
with open('data/renamed-sma_agilent_baits.v7.0.chr_reorderd.bed', 'r') as in_bed_file:
    for bed_entry in in_bed_file:
        chrom, start, stop = bed_entry.rstrip().split("\t")
        for base in range(int(start) - 1, int(stop)):
             accessible_bases[chrom][base]=True


In [ ]:
for pop in pops:
    accessible_genome_size = 0
    pi=0
    td=0
    theta=0
    mu=8.1e-9

    #now loop through each chromosome
    for chrom in list(set(filtered_callset['variants/CHROM'])) :
        target_sites = filtered_callset['variants/CHROM'] == chrom

     
        chr_poss = filtered_callset['variants/POS'][target_sites]
        chr_acs  = pop_ac[pop][target_sites]
        chr_len  = len(accessible_bases[chrom])
        

        chr_pi = allel.sequence_diversity(chr_poss, chr_acs, start=1, stop=chr_len, is_accessible=accessible_bases[chrom])
        chr_theta = allel.watterson_theta(chr_poss, chr_acs, is_accessible=accessible_bases[chrom])
        chr_td = allel.tajima_d(chr_acs, pos=chr_poss, start=1, stop=chr_len, min_sites=3)


        theta = theta + (chr_theta * sum(accessible_bases[chrom]))
        pi = pi + (chr_pi * sum(accessible_bases[chrom]))
        td = td + (chr_td * sum(accessible_bases[chrom]))


        accessible_genome_size = accessible_genome_size + sum(accessible_bases[chrom])

    pi = pi/accessible_genome_size
    td = td/accessible_genome_size
    theta = theta/accessible_genome_size
    ne = theta/(4 * mu)

    outline="{}:{}\t{}\t{}\t{}".format(pop, pi, td, theta, ne)
    print(outline)

#new_world: 0.0006242259807838679 0.0005926896326944684 0.17920916355875174 18292.88989797742
#east_africa: 0.0013018739117935394 0.0016923886667646374 -0.7913682038936288 52234.218110019676
#west_africa: 0.0005014590963616828 0.000960492469167024 -1.7377646136352234 29644.82929527852
#rodhaini: 0.0005310399553706205 0.0004911597410697453 0.29381846808387024 15159.251267584732

other:0.004451787541449815	-0.8256519674458037	0.005518905146039111	170336.57858145403
rodhaini:0.00048278473665803334	0.47865761795155304	0.00042850684952030983	13225.520046923144
